<a href="https://colab.research.google.com/github/aliakbarbadri/natural-language-inference/blob/master/snli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU, TimeDistributed
import unicodedata
import re
import numpy as np
import os
import io
import time

Using TensorFlow backend.


https://www.tensorflow.org/datasets/catalog/snli

https://www.tensorflow.org/addons/tutorials/networks_seq2seq_nmt

https://www.tensorflow.org/tutorials/text/nmt_with_attention

https://nextjournal.com/gkoehler/machine-translation-seq2seq-cpu

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html


# Load data

In [ ]:
ds = tfds.load('snli')

In [3]:
train_ds = ds['train']
valid_ds = ds['validation']
test_ds = ds['test']
#labels: {0: 'entailment', 1: 'neutral', 2: 'contradiction' , -1: '-'}

In [4]:
for example in train_ds.take(1):
  print(example)

{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Washing clothes on a camping trip.'>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'A man washes or dies clothes in a primitive setting.'>}


In [5]:
def preprocess(w):
  def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  w = unicode_to_ascii(w.lower().strip())
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  w = '<start> ' + w + ' <end>'
  return w

In [6]:
# train_data = []
# train_targets = []
# valid_data = []
# valid_targets = []
# test_data = []
# test_targets = []

# for row in train_ds:
#   if int(row["label"]) == 0:
#     train_data.append(preprocess(str(row["hypothesis"].numpy())))
#     train_targets.append(preprocess(str(row["premise"].numpy())))
  

# for row in valid_ds:
#   if int(row["label"]) == 0:
#     valid_data.append(preprocess(str(row["hypothesis"].numpy())))
#     valid_targets.append(preprocess(str(row["premise"].numpy())))

for row in test_ds.take(1):
  if int(row["label"]) == 0:
    print(preprocess(row["hypothesis"].numpy().decode("utf-8")))

    print(preprocess(row["premise"].numpy().decode("utf-8")))

# print("train length", len(train_data))
# print("valid length", len(valid_data))
# print("test length", len(test_data))

<start> a girl is entertaining on stage <end>
<start> a girl in a blue leotard hula hoops on a stage with balloon shapes in the background . <end>


In [7]:
def create_dataset(num_examples):
  pairs = [[preprocess(row["hypothesis"].numpy().decode("utf-8")), preprocess(row["premise"].numpy().decode("utf-8"))]  for row in train_ds.take(num_examples) if int(row["label"])==0]
  return zip(*pairs)

In [109]:
def load_dataset(num_examples=100000):
  input_data, target_data = create_dataset(num_examples)
  
  # input_train, input_val, target_train, target_val = train_test_split(input_data, target_data, test_size=0.2)

  
  tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',oov_token = "<OOV>")
  tokenizer.fit_on_texts(list(set().union(input_data,target_data)))
  
  input_tensor_temp = tokenizer.texts_to_sequences(input_data)
  target_tensor_temp = tokenizer.texts_to_sequences(target_data)

  input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor_temp,padding='post')
  target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor_temp,padding='post')

  return input_tensor, target_tensor, tokenizer

In [110]:
num_examples = 1000
input_tensor, target_tensor, tokenizer = load_dataset(num_examples)
max_length_input, max_length_target  = input_tensor.shape[1], target_tensor.shape[1]
max_length_input, max_length_target

(26, 45)

In [111]:
tf.one_hot(target_tensor[0][1:],vocab_inp_size)

<tf.Tensor: shape=(44, 1228), dtype=float32, numpy=
array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [112]:
target_tensor[0][1:]

array([   2,   32,   13,  283,   12, 1017,   11,  585,  169,    7,    2,
        268,    7,    6,  286,   13,    2,   43,    5,    4,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [113]:
target_tensor_output = tf.one_hot(target_tensor[:,1:],vocab_inp_size)
# target_tensor_output = target_tensor[:,1:]

In [114]:
len(input_tensor), len(target_tensor), len(target_tensor_output)

(362, 362, 362)

In [115]:
def convert(tokenizer, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, tokenizer.index_word[t]))

In [116]:
print ("Input; index to word mapping")
convert(tokenizer, input_tensor[0])
print ()
print ("Target; index to word mapping")
convert(tokenizer, target_tensor[0])

Input; index to word mapping
3 ----> <start>
29 ----> there
11 ----> are
283 ----> motorcycles
7 ----> in
2 ----> a
268 ----> lot
5 ----> .
4 ----> <end>

Target; index to word mapping
3 ----> <start>
2 ----> a
32 ----> group
13 ----> of
283 ----> motorcycles
12 ----> and
1017 ----> scooters
11 ----> are
585 ----> parked
169 ----> together
7 ----> in
2 ----> a
268 ----> lot
7 ----> in
6 ----> the
286 ----> middle
13 ----> of
2 ----> a
43 ----> street
5 ----> .
4 ----> <end>


In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [128]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 32
steps_per_epoch = len(input_tensor)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(tokenizer.word_index)+1
vocab_size = vocab_inp_size
# dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
# dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [129]:
# for e in dataset.take(1):
#   print(len(e[0][0]))
#   print(len(e[1][0]))

In [130]:
input_tensor[0]

array([  3,  29,  11, 283,   7,   2, 268,   5,   4,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [131]:
# example_input_batch, example_target_batch = next(iter(dataset))
# example_input_batch.shape, example_target_batch.shape

# tf example

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 38, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform',
                                   initial_state=enc_output)
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    # self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    # context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    # x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [ ]:
decoder = Decoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),sample_hidden, sample_output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 10653)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  # if (epoch + 1) % 2 == 0:
  #   checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.6436
Epoch 1 Batch 100 Loss 0.6475
Epoch 1 Batch 200 Loss 0.6516
Epoch 1 Batch 300 Loss 0.6314
Epoch 1 Batch 400 Loss 0.6504
Epoch 1 Loss 0.6445
Time taken for 1 epoch 305.2927334308624 sec

Epoch 2 Batch 0 Loss 0.5900
Epoch 2 Batch 100 Loss 0.6335
Epoch 2 Batch 200 Loss 0.6726
Epoch 2 Batch 300 Loss 0.7075
Epoch 2 Batch 400 Loss 0.6723
Epoch 2 Loss 0.6335
Time taken for 1 epoch 305.09351348876953 sec

Epoch 3 Batch 0 Loss 0.6316
Epoch 3 Batch 100 Loss 0.5713
Epoch 3 Batch 200 Loss 0.6167
Epoch 3 Batch 300 Loss 0.6290
Epoch 3 Batch 400 Loss 0.5996
Epoch 3 Loss 0.6258
Time taken for 1 epoch 305.04081320762634 sec



In [ ]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_input, max_length_target))

  sentence = preprocess(sentence)

  inputs = [tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_input,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

  for t in range(max_length_target):
    predictions, dec_hidden = decoder(dec_input,dec_hidden,enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()
    result += tokenizer.index_word[predicted_id] + ' '
    if tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence
    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [ ]:
def translate(sentence):
  result, sentence = evaluate(sentence)
  print('Input: %s' % (sentence))
  print('Predicted: {}'.format(result))

In [ ]:
translate("a girl is entertaining on stage")

Input: <start> a girl is entertaining on stage <end>
Predicted: a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in a man in 
